# Quickdraw Data Preperation

This notebook is to create a test-{sz} and train-{sz} folders full of .png files from .csvs found in the /train folder.

All credit for this work goes to radekosmulski for his inspiration and code.
This code is based on code from a fast.ai MOOC that will be publicly available in Jan 2019

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import cv2
import shutil
import PIL

from fastai import *
from fastai.vision import *

In [2]:
sz = 128 # size
r = 0.1 # portion of images to keep, we want only 1% of total train data

In [3]:
### CAUTION ### The following code will delete any work previously done, be warned!
shutil.rmtree(f'data/train-{sz}-10', ignore_errors=True) # deleting whatever we have saved earlier
os.makedirs(f'data/train-{sz}-10') # best to have where to save the images

In [4]:
# https://www.kaggle.com/gaborfodor/greyscale-mobilenet-lb-0-892
BASE_SIZE = 256
def draw_cv2(raw_strokes, size=256, lw=4, time_color=False):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    img = cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size), interpolation=cv2.INTER_LINEAR)
    else:
        return img

In [5]:
def save_ims_from_df(path):
    df = pd.read_csv(path)
    selected = df[df.recognized==True].sample(int(r * df.shape[0]))
    for row in selected.iterrows():
        idx, drawing, label = row[0], eval(row[1].drawing), '_'.join(row[1].word.split())
        ary = draw_cv2(drawing, size=sz)
        rgb_ary = np.repeat(ary[:,:,None], 3, -1)
        PIL.Image.fromarray(rgb_ary).save(f'data/train-{sz}-10/{label}_{idx}.png')

In [6]:
#%%time
#p = Path('/home/jupyter/tutorials/quickdraw/data/train/')
#i = 1
#for child in p.iterdir():
#    %time save_ims_from_df(child)
#    print(i, "/", len(os.listdir(p)), str(child)[45:])
#    i += 1

In [ ]:
%%time
with ThreadPoolExecutor(22) as e: e.map(save_ims_from_df, Path('data/train').iterdir())

data/train/pear.csv
data/train/golf club.csv
data/train/violin.csv
data/train/flashlight.csv
data/train/swing set.csv
data/train/lighthouse.csv
data/train/broccoli.csvdata/train/sandwich.csv
data/train/purse.csv

data/train/cookie.csv
data/train/fan.csv
data/train/zigzag.csv
data/train/power outlet.csv
data/train/feather.csv
data/train/owl.csv
data/train/radio.csv
data/train/tree.csv
data/train/frog.csv
data/train/popsicle.csv
data/train/streetlight.csv
data/train/peanut.csv
data/train/ladder.csv
data/train/snowflake.csv
data/train/banana.csv
data/train/train.csv
data/train/eraser.csv
data/train/snail.csv
data/train/moon.csv
data/train/flip flops.csv
data/train/barn.csv
data/train/mosquito.csv
data/train/axe.csv
data/train/duck.csv
data/train/The Mona Lisa.csv
data/train/bench.csv
data/train/hockey stick.csv
data/train/grass.csv
data/train/camel.csv
data/train/picture frame.csv
data/train/dragon.csv
data/train/pickup truck.csv
data/train/butterfly.csv
data/train/elbow.csv
data/train/he

Sanity check

In [ ]:
#ls data/train-{sz} -lhS | head -n 10

In [ ]:
# PIL.Image.open(f'data/train-{sz}/camouflage_72469.png')

In [ ]:
df = pd.read_csv('data/test_simplified.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
shutil.rmtree(f'data/test-{sz}', ignore_errors=True)
os.makedirs(f'data/test-{sz}')

In [ ]:
def save_test_ims_from_df(path):
    df = pd.read_csv(path)
    for row in df.iterrows():
        key_id, drawing = row[1].key_id, eval(row[1].drawing)
        ary = draw_cv2(drawing, size=sz)
        rgb_ary = np.repeat(ary[:,:,None], 3, -1)
        PIL.Image.fromarray(rgb_ary).save(f'data/test-{sz}-10/{key_id}.png')

In [ ]:
%time save_test_ims_from_df('data/test_simplified.csv')

In [ ]:
#ls -lht data/test-{sz} | head -n 10

In [ ]:
# PIL.Image.open('data/test-64/9999910529948270.png')